In [1]:
import numpy as np
import pandas as pd

In [306]:
class Node:
    
    def __init__(self, link, children, attribute_name):
        self.link = link
        self.children = children
        self.attribute_name = attribute_name
        
    def add_node(self, node):
        self.children.append(node)

In [311]:
def gain(df, classe, link):
    arr = df[classe].value_counts().values
    p = arr/arr.sum()
    s = sum(-1*p*np.log2(p))
    
    entropy_s = entropyS(df[classe])
    sa = []
    max_gain = ('', -float("inf"))
    for column in df.columns:
        if column != classe:
            gain_value = entropy_s
            for key, value in df[column].value_counts().items():
                gain_value -= get_proportion(df[column])[key][0] * entropyS(df[df[column] == key][classe])
    
            if gain_value > max_gain[1]:
                    max_gain = (column, gain_value)
    
    if df.columns.size <= 1:
        counts = df.iloc[:,0].value_counts()
        return Node(link, [counts.keys()[0]], df.columns[0])
    elif max_gain[1] <= 0:
        counts = df.iloc[:,0].value_counts()
        return Node(link, [counts.keys()[0]], df.columns[0])
    
    node = Node(link, [], max_gain[0])
    for key, value in df[max_gain[0]].value_counts().items():
        children = gain(df[df[max_gain[0]] == key].drop(max_gain[0], axis = 1), classe, key)
        node.add_node(children)
    
    return node

def get_proportion(column):
    counts = column.value_counts()
    counts = counts/counts.sum()
    return pd.DataFrame(counts).pivot_table(columns = counts.keys()).reset_index(drop = True)
    
def entropyS(column):
    arr = column.value_counts().values
    p = arr/arr.sum()
    s = sum(-1*p*np.log2(p))
    return s

def entropySA(column, classe):
    arr = column.value_counts().values
    p = arr/arr.sum()
    s = sum(-1*p*np.log2(p))
    return s

In [312]:
d = {'Outlook': ['Sunny','Sunny','Overcast','Rain','Rain','Rain','Overcast','Sunny','Sunny','Rain','Sunny','Overcast','Overcast','Rain'], 'Temperature': ['Hot','Hot','Hot','Mild','Cool','Cool','Cool','Mild','Cool','Mild','Mild','Mild','Hot','Mild'], 'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'], 'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Strong'], 'Decision': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']}
df = pd.DataFrame(d)

In [313]:
tree = gain(df, 'Decision', 'Root')

In [316]:
tree.attribute_name

'Outlook'

In [248]:
df['Outlook'].value_counts().sum()

14